# Detecting malicious URLs
Some introduction...

## Data exploration

In [2]:
import os
import pandas as pd
import numpy as np
from urllib.parse import urlparse

In [3]:
DATASET_PATH = 'datasets/malicious_urls.csv'

In [4]:
df = pd.read_csv(DATASET_PATH)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 651191 entries, 0 to 651190
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   url     651191 non-null  object
 1   type    651191 non-null  object
dtypes: object(2)
memory usage: 9.9+ MB


In [8]:
df.describe(include='all')

,url,type
count,651191,651191
unique,641119,4
top,http://style.org.hc360.com/css/detail/mysite/s...,benign
freq,180,428103
